In [3]:
import pygame
import random
import time
import csv
import sys
import io
import matplotlib.pyplot as plt
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas as pdfcanvas
from reportlab.lib.utils import ImageReader

# ----- CONFIGURATION -----
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 600
BG_COLOR = (240, 240, 240)

CIRCLE_RADIUS = 50
SMALL_RADIUS = 20

BUTTON_W, BUTTON_H = 120, 50
LEFT_CENTER = (200, 150)
RIGHT_CENTER = (600, 150)
LEFT_BTN_RECT = pygame.Rect(
    LEFT_CENTER[0] - BUTTON_W//2,
    LEFT_CENTER[1] + CIRCLE_RADIUS + 30,
    BUTTON_W, BUTTON_H
)
RIGHT_BTN_RECT = pygame.Rect(
    RIGHT_CENTER[0] - BUTTON_W//2,
    RIGHT_CENTER[1] + CIRCLE_RADIUS + 30,
    BUTTON_W, BUTTON_H
)

FIXATION_TIME = 0.5  # seconds
STIM_DURATION = 1.5  # seconds
ITI_MIN, ITI_MAX = 1.0, 2.0

PRACTICE_TRIALS = 5   # practice trials
NUM_TRIALS = 40       # main trials
DISTRACTOR_P = 0.2    # probability of distractor inner circle

DATA_FILE = 'ior_reaction_times.csv'
REPORT_FILE = 'ior_report.pdf'
# --------------------------

def log_header():
    with open(DATA_FILE, 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(['trial', 'side', 'is_target', 'rt', 'response', 'accuracy'])

def log_trial(trial, side, is_target, rt, response, accuracy):
    with open(DATA_FILE, 'a', newline='') as f:
        writer = csv.writer(f)
        writer.writerow([
            trial,
            side,
            int(is_target),
            round(rt, 3) if rt is not None else '',
            response,
            accuracy
        ])

def show_vector_check(screen, side):
    """Draws a thick green vector checkmark next to the correct button."""
    if side == 'left':
        base_x = LEFT_BTN_RECT.right + 20
        base_y = LEFT_BTN_RECT.centery
    else:
        base_x = RIGHT_BTN_RECT.x - 20
        base_y = RIGHT_BTN_RECT.centery

    # two-line check shape
    p1 = (base_x, base_y)
    p2 = (base_x + 12, base_y + 18)
    p3 = (base_x + 36, base_y - 12)
    pygame.draw.lines(screen, (0, 200, 0), False, [p1, p2, p3], 6)
    pygame.display.flip()
    time.sleep(0.5)

def generate_pdf_report():
    # 1) Read data
    hits = misses = false_alarms = correct_inhibitions = incorrect_buttons = 0
    rts = []
    with open(DATA_FILE) as f:
        reader = csv.DictReader(f)
        for row in reader:
            acc = row['accuracy']
            if acc == 'hit':
                hits += 1
                rts.append(float(row['rt']))
            elif acc == 'miss':
                misses += 1
            elif acc == 'false_alarm':
                false_alarms += 1
            elif acc == 'correct_inhibit':
                correct_inhibitions += 1
            elif acc == 'incorrect_button':
                incorrect_buttons += 1

    go_trials = hits + misses
    total_mistakes = misses + false_alarms + incorrect_buttons
    total_trials = hits + misses + false_alarms + correct_inhibitions + incorrect_buttons
    accuracy_pct = (hits / go_trials * 100) if go_trials else 0
    mean_rt = (sum(rts) / len(rts)) if rts else 0

    # 2) Plot
    categories = ['Accuracy (%)', 'Mistakes', 'Mean RT (s)']
    values     = [accuracy_pct, total_mistakes, mean_rt]
    fig, ax = plt.subplots(figsize=(6, 3))
    bars = ax.bar(categories, values)
    ax.set_title('Overall Performance Metrics')
    ax.set_ylabel('Value')
    for bar, val in zip(bars, values):
        ax.text(bar.get_x() + bar.get_width()/2,
                val + max(values)*0.02,
                f'{val:.1f}', ha='center', va='bottom')
    fig.tight_layout()

    buf = io.BytesIO()
    fig.savefig(buf, format='PNG', dpi=150)
    buf.seek(0)
    plt.close(fig)
    img_reader = ImageReader(buf)

    # 3) Build PDF
    c = pdfcanvas.Canvas(REPORT_FILE, pagesize=letter)
    c.setFont("Helvetica-Bold", 18)
    c.drawString(50, 750, "Inhibition of Return Test Report")
    c.setFont("Helvetica", 12)
    lines = [
        f"Total trials:               {total_trials}",
        f"Go trials (green target):   {go_trials}",
        f"  • Hits:                   {hits}",
        f"  • Misses:                 {misses}",
        f"No-go trials (distractor):  {correct_inhibitions + false_alarms}",
        f"  • Correct inhibits:       {correct_inhibitions}",
        f"  • False alarms:           {false_alarms}",
        f"Incorrect-button presses:   {incorrect_buttons}",
        f"Mean RT (hits only):        {mean_rt:.3f} s",
        f"Accuracy (go trials):       {accuracy_pct:.1f} %",
        f"Total mistakes:             {total_mistakes}",
    ]
    y = 710
    for line in lines:
        c.drawString(60, y, line)
        y -= 18

    c.drawImage(img_reader, x=60, y=350, width=480, height=200, mask='auto')
    c.showPage()
    c.save()
    print(f"PDF report saved to {REPORT_FILE}")

def draw_header(screen, font, trial, total):
    title = font.render("Inhibition of Return Test", True, (50, 50, 50))
    screen.blit(title, (SCREEN_WIDTH//2 - title.get_width()//2, 10))
    # progress bar
    bar_w = int(SCREEN_WIDTH * 0.8)
    bar_h = 20
    bar_x = int(SCREEN_WIDTH * 0.1)
    bar_y = 60
    pygame.draw.rect(screen, (180, 180, 180), (bar_x, bar_y, bar_w, bar_h), 2)
    progress = (trial - 1) / total
    fill = int(bar_w * progress)
    if fill > 0:
        pygame.draw.rect(screen, (0, 200, 0), (bar_x+2, bar_y+2, fill, bar_h-4))
    txt = font.render(f"{trial}/{total}", True, (50, 50, 50))
    screen.blit(txt, (bar_x + bar_w + 10, bar_y - bar_h//2))

def run_trials(screen, font, title_font, num_trials, practice=False):
    for trial in range(1, num_trials + 1):
        # 1) fixation
        screen.fill(BG_COLOR)
        draw_header(screen, title_font, trial, num_trials)
        pygame.draw.circle(screen, (200,200,200), LEFT_CENTER, CIRCLE_RADIUS)
        pygame.draw.circle(screen, (200,200,200), RIGHT_CENTER, CIRCLE_RADIUS)
        pygame.draw.circle(screen, (180,180,180), LEFT_CENTER, SMALL_RADIUS)
        pygame.draw.circle(screen, (180,180,180), RIGHT_CENTER, SMALL_RADIUS)
        pygame.draw.rect(screen, (100,100,100), LEFT_BTN_RECT, border_radius=8)
        pygame.draw.rect(screen, (100,100,100), RIGHT_BTN_RECT, border_radius=8)
        lblL = font.render("Left", True, (255,255,255))
        lblR = font.render("Right", True, (255,255,255))
        screen.blit(lblL, lblL.get_rect(center=LEFT_BTN_RECT.center))
        screen.blit(lblR, lblR.get_rect(center=RIGHT_BTN_RECT.center))
        pygame.display.flip()
        time.sleep(FIXATION_TIME)

        # 2) setup stimulus
        side = random.choice(['left','right'])
        is_target = random.random() > DISTRACTOR_P
        start = time.time()
        responded = False
        rt = None
        resp = 'none'

        # 3) present & collect
        stim_on = time.time()
        while time.time() - stim_on < STIM_DURATION:
            for e in pygame.event.get():
                if e.type == pygame.QUIT:
                    pygame.quit(); sys.exit()
                if e.type == pygame.MOUSEBUTTONDOWN and not responded:
                    if LEFT_BTN_RECT.collidepoint(e.pos):
                        responded = True; resp='left'; rt = time.time()-start
                    elif RIGHT_BTN_RECT.collidepoint(e.pos):
                        responded = True; resp='right'; rt = time.time()-start
            if responded: break

            # redraw
            screen.fill(BG_COLOR)
            draw_header(screen, title_font, trial, num_trials)
            bigL = (0,200,0) if is_target and side=='left' else (200,200,200)
            bigR = (0,200,0) if is_target and side=='right' else (200,200,200)
            smlL = (200,200,0) if not is_target and side=='left' else (180,180,180)
            smlR = (200,200,0) if not is_target and side=='right' else (180,180,180)
            pygame.draw.circle(screen, bigL, LEFT_CENTER, CIRCLE_RADIUS)
            pygame.draw.circle(screen, bigR, RIGHT_CENTER, CIRCLE_RADIUS)
            pygame.draw.circle(screen, smlL, LEFT_CENTER, SMALL_RADIUS)
            pygame.draw.circle(screen, smlR, RIGHT_CENTER, SMALL_RADIUS)
            pygame.draw.rect(screen, (100,100,100), LEFT_BTN_RECT, border_radius=8)
            pygame.draw.rect(screen, (100,100,100), RIGHT_BTN_RECT, border_radius=8)
            screen.blit(lblL, lblL.get_rect(center=LEFT_BTN_RECT.center))
            screen.blit(lblR, lblR.get_rect(center=RIGHT_BTN_RECT.center))
            pygame.display.flip()

        # 4) accuracy
        if is_target:
            if responded and resp==side: acc='hit'
            elif responded: acc='incorrect_button'
            else: acc='miss'
        else:
            acc = 'false_alarm' if responded else 'correct_inhibit'
            rt = None

        # 5) practice feedback only
        if practice and acc=='hit':
            # redraw so check is on top
            screen.fill(BG_COLOR)
            draw_header(screen, title_font, trial, num_trials)
            bigL = (0,200,0) if side=='left' else (200,200,200)
            bigR = (0,200,0) if side=='right' else (200,200,200)
            pygame.draw.circle(screen, bigL, LEFT_CENTER, CIRCLE_RADIUS)
            pygame.draw.circle(screen, bigR, RIGHT_CENTER, CIRCLE_RADIUS)
            pygame.draw.rect(screen, (100,100,100), LEFT_BTN_RECT, border_radius=8)
            pygame.draw.rect(screen, (100,100,100), RIGHT_BTN_RECT, border_radius=8)
            screen.blit(lblL, lblL.get_rect(center=LEFT_BTN_RECT.center))
            screen.blit(lblR, lblR.get_rect(center=RIGHT_BTN_RECT.center))
            show_vector_check(screen, side)

        # 6) log main trials
        if not practice:
            log_trial(trial, side, is_target, rt, resp, acc)

        # 7) ITI
        time.sleep(random.uniform(ITI_MIN, ITI_MAX))

def draw_practice_menu(screen, font):
    screen.fill(BG_COLOR)
    btn_w, btn_h = 200, 60
    p_btn = pygame.Rect(SCREEN_WIDTH//2 - btn_w - 20, SCREEN_HEIGHT//2, btn_w, btn_h)
    s_btn = pygame.Rect(SCREEN_WIDTH//2 + 20, SCREEN_HEIGHT//2, btn_w, btn_h)
    pygame.draw.rect(screen, (100,100,100), p_btn, border_radius=8)
    pygame.draw.rect(screen, (100,100,100), s_btn, border_radius=8)

    # Practice Again text
    txt_pr = font.render("Practice Again", True, (255,255,255))
    rect_pr = txt_pr.get_rect(center=p_btn.center)
    screen.blit(txt_pr, rect_pr)

    # Start Main Game text
    txt_st = font.render("Start Main Game", True, (255,255,255))
    rect_st = txt_st.get_rect(center=s_btn.center)
    screen.blit(txt_st, rect_st)

    pygame.display.flip()
    return p_btn, s_btn

def main():
    pygame.init()
    screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT))
    pygame.display.set_caption("Inhibition of Return Test")
    font = pygame.font.SysFont(None, 32)
    title_font = pygame.font.SysFont(None, 36, bold=True)

    log_header()

    # practice
    run_trials(screen, font, title_font, PRACTICE_TRIALS, practice=True)

    # practice menu
    while True:
        pr_btn, st_btn = draw_practice_menu(screen, font)
        for e in pygame.event.get():
            if e.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if e.type == pygame.MOUSEBUTTONDOWN:
                if pr_btn.collidepoint(e.pos):
                    run_trials(screen, font, title_font, PRACTICE_TRIALS, practice=True)
                elif st_btn.collidepoint(e.pos):
                    run_trials(screen, font, title_font, NUM_TRIALS, practice=False)
                    pygame.quit()
                    print(f"Done — data in {DATA_FILE}")
                    generate_pdf_report()
                    sys.exit()

if __name__ == '__main__':
    main()

Done — data in ior_reaction_times.csv
PDF report saved to ior_report.pdf


SystemExit: 